<a href="https://colab.research.google.com/github/stelmak/llm-hallucinations/blob/main/%D0%B0%D0%BD%D0%B8%D0%B3%D0%B0%D0%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install openpyxl
!pip install cloudscraper
!pip install xlsxwriter

import requests
from bs4 import BeautifulSoup
import pandas as pd
import cloudscraper
import os

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 13.4 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.7 MB/s eta 0:00:00


In [ ]:
from groq import Groq
api_key = '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'
client = Groq(api_key=api_key)

In [ ]:
models = client.models.list()
print(models)

ModelListResponse(data=[Model(id='llama-guard-3-8b', created=1693721698, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None, max_completion_tokens=8192), Model(id='allam-2-7b', created=1737672203, object='model', owned_by='SDAIA', active=True, context_window=4096, public_apps=None, max_completion_tokens=4096), Model(id='llama3-8b-8192', created=1693721698, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None, max_completion_tokens=8192), Model(id='meta-llama/llama-4-maverick-17b-128e-instruct', created=1743877158, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=8192), Model(id='llama-3.3-70b-versatile', created=1733447754, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=32768), Model(id='playai-tts-arabic', created=1740682783, object='model', owned_by='PlayAI', active=True, context_window=8192, public_a

In [ ]:
def llm_inference_statements(theme):
  output = client.chat.completions.create(messages=[
                                                              {
                                                                 # И в системной, и в основной роли можно использовать f-strings
                                                                  "role": "system", # Указываем, что это системная роль, эту строку не меняем
                                                                  "content": "generate json's, no explanation or additional text, use the following structure:\n"
                                                                              "statements: []\n"
                                                                              "Array must contain exactly 10 only true statemnets on a given topic"
                                                                              'No additional verification. No true or false. Only statements'

                                                              },
                                                              {
                                                                  "role": "user", # Указываем, что это текущий промпт
                                                                  "content": f"synthesize data on {theme}"} # Меняем содержание контента: это может быть переменная, например, из фукнции
                                                    ],
                                                          #форматирование ответа
                                                          response_format=
                                                          {
                                                              "type": "json_object"
                                                          },
                                                    model='llama-3.1-8b-instant', # Тип модели
                                                    stream=False,
                                                    max_tokens=1024, # Максимальная длина выдачи, желательно поменять под задачу
                                                    temperature=0.5, # Температура
                                                    top_p=0.1 # Топ-P отвечает за разнообразие выдачи
                                                          ).choices[0].message.content
  return output

In [ ]:
def llm_inference_questinos(theme):
  output = client.chat.completions.create(messages=[
                                                              {
                                                                 # И в системной, и в основной роли можно использовать f-strings
                                                                  "role": "system", # Указываем, что это системная роль, эту строку не меняем
                                                                  "content": "generate json's, no explanation or additional text, use the following structure:\n"
                                                                              "questions: []\n"
                                                                              "Array must contain exactly 10 questions based on 10 statements that were generated earlier"
                                                                              'Generate only questions, no answers and no ids'
                                                              },
                                                              {
                                                                  "role": "user", # Указываем, что это текущий промпт
                                                                  "content": f"synthesize data on {theme}"} # Меняем содержание контента: это может быть переменная, например, из фукнции
                                                    ],
                                                          # Форматирование ответа
                                                          response_format=
                                                          {
                                                              "type": "json_object"
                                                          },
                                                    model='llama-3.1-8b-instant', # Тип модели
                                                    stream=False,
                                                    max_tokens=1024, # Максимальная длина выдачи, желательно поменять под задачу
                                                    temperature=0.5, # Температура
                                                    top_p=0.1 # Топ-P отвечает за разнообразие выдачи
                                                          ).choices[0].message.content
  return output

In [ ]:
def llm_inference_questinos_answer(theme):
  output = client.chat.completions.create(messages=[
                                                              {
                                                                 # И в системной, и в основной роли можно использовать f-strings
                                                                  "role": "system", # Указываем, что это системная роль, эту строку не меняем
                                                                  "content": "generate json's, no explanation or additional text, use the following structure:\n"
                                                                              "answer_to_the_questions: []\n"
                                                                              "Array must contain exactly 10 answers on 10 given questions that were generated earlier"
                                                                              'Generate only answers, no additional information, no questions'
                                                              },
                                                              {
                                                                  "role": "user", # Указываем, что это текущий промпт
                                                                  "content": f"synthesize data on {theme}"} # Меняем содержание контента: это может быть переменная, например, из фукнции
                                                    ],
                                                          # Форматирование ответа
                                                          response_format=
                                                          {
                                                              "type": "json_object"
                                                          },
                                                    model='llama-3.1-8b-instant', # Тип модели
                                                    stream=False,
                                                    max_tokens=1024, # Максимальная длина выдачи, желательно поменять под задачу
                                                    temperature=0.5, # Температура
                                                    top_p=0.1 # Топ-P отвечает за разнообразие выдачи
                                                          ).choices[0].message.content
  return output

In [ ]:
def llm_inference_contradicting_statements(theme):
  output = client.chat.completions.create(messages=[
                                                              {
                                                                 # И в системной, и в основной роли можно использовать f-strings
                                                                  "role": "system", # Указываем, что это системная роль, эту строку не меняем
                                                                  "content": "generate json's, no explanation or additional text, use the following structure:\n"
                                                                              "contradicting_statements: []"
                                                                              "Array must contain exactly 10 entries that contradict 10 statements that were generated earlier. Only statements without verification"
                                                              },
                                                              {
                                                                  "role": "user", # Указываем, что это текущий промпт
                                                                  "content": f"synthesize data on {theme}"} # Меняем содержание контента: это может быть переменная, например, из фукнции
                                                    ],
                                                          # Форматирование ответа
                                                          response_format=
                                                          {
                                                              "type": "json_object"
                                                          },
                                                    model='llama-3.1-8b-instant', # Тип модели
                                                    stream=False,
                                                    max_tokens=1024, # Максимальная длина выдачи, желательно поменять под задачу
                                                    temperature=0.5, # Температура
                                                    top_p=0.1 # Топ-P отвечает за разнообразие выдачи
                                                          ).choices[0].message.content
  return output

In [ ]:
def llm_inference_contradicting_statements_verification(theme):
  output = client.chat.completions.create(messages=[
                                                              {
                                                                 # И в системной, и в основной роли можно использовать f-strings
                                                                  "role": "system", # Указываем, что это системная роль, эту строку не меняем
                                                                  "content": "generate json's, no explanation or additional text, use the following structure:\n"
                                                                              "contradicting_statements_verification: []"
                                                                              "Array must contain exactly 10 entries. Only verification: 'True' or 'False'. Do not generate any full statement, only 'True' or 'False'"
                                                              },
                                                              {
                                                                  "role": "user", # Указываем, что это текущий промпт
                                                                  "content": f"synthesize data on {theme}"} # Меняем содержание контента: это может быть переменная, например, из фукнции
                                                    ],
                                                          # Форматирование ответа
                                                          response_format=
                                                          {
                                                              "type": "json_object"
                                                          },
                                                    model='llama-3.1-8b-instant', # Тип модели
                                                    stream=False,
                                                    max_tokens=1024, # Максимальная длина выдачи, желательно поменять под задачу
                                                    temperature=0.5, # Температура
                                                    top_p=0.1 # Топ-P отвечает за разнообразие выдачи
                                                          ).choices[0].message.content
  return output

In [ ]:
def remove_from_html(al):
  soup = BeautifulSoup(al, "lxml")
  content_div = soup.find("div", class_="mw-parser-output")
  if content_div:
      for a_tag in content_div.find_all("a"):
          a_tag.unwrap()
      for info in content_div.find_all("div", role='note'):
          info.decompose()
      for info_1 in content_div.find_all("table", class_="infobox"):
          info_1.decompose()
      for info_2 in content_div.find_all("div", class_="mw-heading mw-heading2"):
        if 'См. также' in info_2.get_text():
          info_2.decompose()
        elif 'Ссылки' in info_2.get_text():
          info_2.decompose()
        elif 'Примечания' in info_2.get_text():
          info_2.decompose()
        elif 'Литература' in info_2.get_text():
          info_2.decompose()
        elif 'Ссылки' in info_2.get_text():
          info_2.decompose()
      for info_3 in content_div.find_all("div", class_="columns"):
          info_3.decompose()
      for info_4 in content_div.find_all("span", class_="mw-editsection"):
          info_4.decompose()
      for info_5 in content_div.find_all("div", class_="toc"):
          info_5.decompose()
      for info_6 in content_div.find_all("div", role='navigation', class_="navbox"):
          info_6.decompose()
      for info_7 in content_div.find_all("div", class_="mw-normal-catlinks"):
          info_7.decompose()
      for info_8 in content_div.find_all("div", class_="collapseRefs"):
          info_8.decompose()
      for info_9 in content_div.find_all("ul"):
          info_9.decompose()
      for info_10 in content_div.find_all("dl"):
          info_10.decompose()
      for info_11 in content_div.find_all("div", class_="ts-Родственные_проекты ruwikiWikimediaNavigation metadata plainlinks plainlist noprint"):
          info_11.decompose()
      for info_12 in content_div.find_all("sup", class_='reference'):
          info_12.decompose()
      for info_13 in content_div.find_all("figcaption"):
          info_13.decompose()
      for info_14 in content_div.find_all("table", class_="plainlinks metadata ambox ambox-style ambox-multiple_issues"):
          info_14.decompose()
      return content_div.get_text()
  else:
        return "Контент не найден"


In [ ]:
themes = ['Наука', 'Физика', 'Химия', 'Биология', 'Нейробиология', 'Зоология', 'Ботаника', 'Математика', 'Статистика', 'Астрономия', 'География', 'Геология', 'Экология', 'Энергетика', 'Психология', 'Социология', 'Педагогика', 'Образование', 'Философия', 'Логика', 'История', 'Археология', 'Антропология', 'Этнография', 'Лингвистика', 'Семиотика', 'Религиоведение', 'Астрология', 'Информационные технологии', 'Компьютерные технологии', 'Биотехнология', 'Нанотехнология', 'Робототехника', 'Квантовые вычисления', 'Генетика', 'Генная инженерия', 'Автоматизация', 'Механика', 'Машиностроение', 'Информатика', 'Электротехника', 'Электроника', 'Телекоммуникации', 'Интернет вещей', 'Искусственный интеллект', 'Кибербезопасность', 'Облачные вычисления', 'Большие данные', 'Строительство', 'Сельское хозяйство', 'Общество', 'Культурология', 'Культура', 'Цивилизация', 'Глобализация', 'Миграция', 'Картография', 'Общественные науки', 'Гендерные исследования', 'Социальная психология', 'Журналистика', 'Политология', 'Право', 'Криминология', 'Экономика', 'Финансы', 'Бизнес', 'Маркетинг', 'Менеджмент', 'Предпринимательство', 'Устойчивое развитие', 'Экологическая экономика', 'Социальное предпринимательство', 'Гуманитарные науки', 'Филология', 'Искусствоведение', 'Литература', 'Литературоведение', 'Cпорт', 'Музыка', 'Живопись', 'Скульптура', 'Театр', 'Кино', 'Фотография', 'Архитектура', 'Дизайн', 'Мода', 'Поэзия', 'Проза', 'Драматургия', 'Иллюстрация', 'Анимация', 'Графический дизайн', 'Индустриальный дизайн', 'Интерьер', 'Ландшафтная архитектура', 'Медицина', 'Стоматология', 'Педиатрия', 'Психиатрия', 'Ветеринария']

In [ ]:
print(len(themes))

102


In [ ]:
datasets = []
for theme in themes:
  new_url = 'https://ru.wikipedia.org/wiki/' + theme
  req = requests.get(new_url)
  al = req.text
  clean_text = remove_from_html(al)
  short_text = clean_text[:6000]
  datasets.append(short_text)

In [ ]:
c = 0
for idx, i in enumerate(datasets):
    if 'Контент не найден' in i:
      c += 1
      print(themes[idx])
      print(idx)
print(c)

In [ ]:
new_dataset = []
new_url = 'https://ru.wikipedia.org/wiki/Спорт'
req = requests.get(new_url)
al = req.text
clean_text = remove_from_html(al)
short_text = clean_text[:6000]
datasets[78] = short_text

In [ ]:
import json

In [ ]:
with open('example.json', 'w') as file:
  json.dump(datasets, file)

from google.colab import files
files.download('example.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving example.json to example (2).json


In [ ]:
with open("example (2).json", "r") as f:
    dataset = json.load(f)

print(dataset)

['\n\nНау́ка\xa0(праслав. *na- + *učiti\xa0— учить, выкнуть)\xa0— деятельность, направленная на выработку и систематизацию объективных знаний о действительности.\nЭта деятельность осуществляется путём сбора фактов, их регулярного обновления, систематизации и критического анализа. На этой основе выполняется обобщение или синтез новых знаний, которые описывают наблюдаемые природные или общественные явления и указывают на причинно-следственные связи, что позволяет осуществлять прогнозирование. Те гипотезы, которые описывают совокупность наблюдаемых фактов и не опровергаются экспериментами, признаются законами природы или общества (см. научный метод).\nПо мнению историка науки И.\xa0Н.\xa0Веселовского, «Современная наука, по существу, опирается на следующие три основы: 1) применение эксперимента, наблюдения и опыта при изучении природы; 2) логические доказательства выводов, полученных из основных предпосылок; 3) возможность математического представления природных процессов».\nНаука в широк

In [ ]:
print(dataset[15])

In [ ]:
print(dataset[55])

NameError: name 'dataset' is not defined

In [ ]:
print(dataset[71])

42°16′20″ с. ш. 71°04′13″ з. д.HGЯO


In [ ]:
print(dataset[72])

In [ ]:
print(dataset[67])

In [ ]:
print(dataset[15])

In [ ]:
c = -1
outputs_statements = []
outputs_contradicting_statements = []
outputs_questions = []

for text in dataset:
  outputs_statements.append(llm_inference_statements(text))
  c += 1
  print(c)

for statement in outputs_statements:
  outputs_questions.append(llm_inference_questinos(statement))
  outputs_contradicting_statements.append(llm_inference_contradicting_statements(statement))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


In [ ]:
outputs_contradicting_statements_veryfication = []
outputs_questions_answers = []

for question in outputs_questions:
  outputs_questions_answers.append(llm_inference_questinos_answer(question))

for statement in outputs_contradicting_statements:
  outputs_contradicting_statements_veryfication.append(llm_inference_contradicting_statements_verification(statement))

In [ ]:
import ast

In [ ]:
outputs_statements_1 = []
outputs_statements_1.append(llm_inference_statements(dataset[0]))
print(outputs_statements_1)

['{\n   "statements": [\n      "Религиоведение - академическая область научных исследований, посвященная исследованию религиозных убеждений, поведения и институтов.",\n      "Религиоведение описывает, сравнивает, интерпретирует и объясняет религию, уделяя особое внимание систематическим, исторически обоснованным и межкультурным перспективам.",\n      "Предметом религиоведения являются все существовавшие в прошлом и существующие ныне религии.",\n      "Религиоведение пытается изучать религиозное поведение и убеждения вне какой-либо конкретной религиозной точки зрения.",\n      "Религиоведение опирается на множество дисциплин и их методы, включая антропологию, социологию, психологию, философию и историю религии.",\n      "Религиоведение изучает закономерности возникновения, развития и функционирования религий, их строение и различные компоненты, многообразные феномены в истории общества.",\n      "Религиоведение исследует все религии в равной мере - мировые религии, этнические религии, с

In [ ]:
outputs_contradicting_statements_1 = []
outputs_contradicting_statements_1.append(llm_inference_contradicting_statements(outputs_statements_1[0]))

In [ ]:
outputs_questions_1 = []
outputs_questions_1.append(llm_inference_questinos(outputs_statements_1[0]))

outputs_questions_answers_1 = []
outputs_questions_answers_1.append(llm_inference_questinos_answer(outputs_questions_1[0]))

In [ ]:
outputs_contradicting_statements_veryfication_1 = []
outputs_contradicting_statements_veryfication_1.append(llm_inference_contradicting_statements_verification(outputs_contradicting_statements_1[0]))
print(outputs_questions_answers_1)
print(outputs_contradicting_statements_veryfication_1)

['{\n   "answer_to_the_questions": [\n      "Религиоведение",\n      "Описание, сравнение, интерпретация и объяснение",\n      "Христианство, ислам, буддизм, индуизм, иудаизм и другие",\n      "Объяснение религиозного поведения и убеждений",\n      "Антропология, социология, история, философия, лингвистика и другие",\n      "Закономерности религиозного поведения, убеждений и институтов",\n      "Все религии",\n      "XIX век",\n      "Университет в США",\n      "Отличается по акценту на научном подходе и отсутствии религиозной приверженности"\n   ]\n}']
['{\n   "contradicting_statements_verification": [\n      "False",\n      "False",\n      "False",\n      "False",\n      "False",\n      "False",\n      "False",\n      "False",\n      "False",\n      "False"\n   ]\n}']


In [ ]:
print(outputs_contradicting_statements_1)
print(outputs_statements_1)

['{\n   "contradicting_statements": [\n      "Религиоведение - академическая область научных исследований, посвященная исследованию религиозных убеждений, поведения и институтов, но только для христиан.",\n      "Религиоведение описывает, сравнивает, интерпретирует и объясняет религию, но только с точки зрения западной культуры.",\n      "Предметом религиоведения являются только существующие ныне религии, а не прошлые.",\n      "Религиоведение пытается изучать религиозное поведение и убеждения только с конкретной религиозной точки зрения.",\n      "Религиоведение опирается только на социологию и историю религии, а не на антропологию, психологию и философию.",\n      "Религиоведение изучает только закономерности возникновения и развития религий, а не их функционирования и строения.",\n      "Религиоведение исследует только мировые религии, а не этнические религии и современные религиозные движения.",\n      "Религиоведение является старой академической дисциплиной, которая начала формир

In [ ]:
print(len(outputs_contradicting_statements_1))
print(len(outputs_statements_1))
print(len(outputs_questions_1))
print(len(outputs_questions_answers_1))
print(len(outputs_contradicting_statements_veryfication_1))

1
1
1
1
1


In [ ]:
final_dataframe = {
    'statements': [],
    'contradicting_statements': [],
    'verification': [],
    'questions': [],
    'answers':[]
}

for i in range(len(outputs_statements)):
    sample_statements = outputs_statements[i]
    sample_statements = sample_statements.replace('json', '').replace('```', '').strip()

    try:
        sample_changed_statemnets = ast.literal_eval(sample_statements)
    except Exception as e:
        print(f"Ошибка при обработке sample[{i}]: {e}")
        continue

    for j in range(len(sample_changed_statemnets['statements'])):
        final_dataframe['statements'].append(sample_changed_statemnets['statements'][j])

for i in range(len(outputs_contradicting_statements)):
    sample_cr_statements = outputs_contradicting_statements[i]
    sample_cr_statements = sample_cr_statements.replace('json', '').replace('```', '').strip()

    try:
        sample_changed_cr_statements = ast.literal_eval(sample_cr_statements)
    except Exception as e:
        print(f"Ошибка при обработке sample[{i}]: {e}")
        continue
    for j in range(len(sample_changed_cr_statements['contradicting_statements'])):
        final_dataframe['contradicting_statements'].append(sample_changed_cr_statements['contradicting_statements'][j])

for i in range(len(outputs_questions)):
    sample_questions = outputs_questions[i]
    sample_questions = sample_questions.replace('json', '').replace('```', '').strip()

    try:
        sample_changed_questions = ast.literal_eval(sample_questions)
    except Exception as e:
        print(f"Ошибка при обработке sample[{i}]: {e}")
        continue
    for j in range(len(sample_changed_questions['questions'])):
        final_dataframe['questions'].append(sample_changed_questions['questions'][j])

for i in range(len(outputs_questions_answers)):
    sample_answers = outputs_questions_answers[i]
    sample_answers = sample_answers.replace('json', '').replace('```', '').strip()

    try:
        sample_changed_answers = ast.literal_eval(sample_answers)
    except Exception as e:
        print(f"Ошибка при обработке sample[{i}]: {e}")
        continue

    for j in range(len(sample_changed_answers['answer_to_the_questions'])):
        final_dataframe['answers'].append(sample_changed_answers['answer_to_the_questions'][j])

for i in range(len(outputs_contradicting_statements_veryfication)):
    sample_verification = outputs_contradicting_statements_veryfication[i]
    sample_verification = sample_verification.replace('json', '').replace('```', '').strip()

    try:
        sample_changed_verification = ast.literal_eval(sample_verification)
    except Exception as e:
        print(f"Ошибка при обработке sample[{i}]: {e}")
        continue

    for j in range(len(sample_changed_verification['contradicting_statements_verification'])):
        final_dataframe['verification'].append(sample_changed_verification['contradicting_statements_verification'][j])

Ошибка при обработке sample[71]: malformed node or string on line 5: <ast.Name object at 0x7fc69f9bc3a0>


In [ ]:
final_dataframe = {
    'statements': [],
    'contradicting_statements': [],
    'questions': []
}

for i in range(len(outputs_statements)):
    sample_statements = outputs_statements[i]
    sample_statements = sample_statements.replace('json', '').replace('```', '').strip()

    try:
        sample_changed_statemnets = ast.literal_eval(sample_statements)
    except Exception as e:
        print(f"Ошибка при обработке sample[{i}]: {e}")
        continue

    for j in range(len(sample_changed_statemnets['statements'])):
        final_dataframe['statements'].append(sample_changed_statemnets['statements'][j])

for i in range(len(outputs_contradicting_statements)):
    sample_cr_statements = outputs_contradicting_statements[i]
    sample_cr_statements = sample_cr_statements.replace('json', '').replace('```', '').strip()

    try:
        sample_changed_cr_statements = ast.literal_eval(sample_cr_statements)
    except Exception as e:
        print(f"Ошибка при обработке sample[{i}]: {e}")
        continue
    for j in range(len(sample_changed_cr_statements['contradicting_statements'])):
        final_dataframe['contradicting_statements'].append(sample_changed_cr_statements['contradicting_statements'][j])

for i in range(len(outputs_questions)):
    sample_questions = outputs_questions[i]
    sample_questions = sample_questions.replace('json', '').replace('```', '').strip()

    try:
        sample_changed_questions = ast.literal_eval(sample_questions)
    except Exception as e:
        print(f"Ошибка при обработке sample[{i}]: {e}")
        continue
    for j in range(len(sample_changed_questions['questions'])):
        final_dataframe['questions'].append(sample_changed_questions['questions'][j])

In [ ]:
import pandas as pd

In [ ]:
target_length = 1000

for key in final_dataframe:
    current = final_dataframe[key]
    length = len(current)

    if length > target_length:
        final_dataframe[key] = current[:target_length]

In [ ]:
print(len(final_dataframe['questions']))
print(len(final_dataframe['contradicting_statements']))
print(len(final_dataframe['statements']))
print(len(final_dataframe['answers']))
print(len(final_dataframe['verification']))

1000
1000
1000
1000
1000


In [ ]:
df = pd.DataFrame(final_dataframe)

In [ ]:
df

,statements,contradicting_statements,verification,questions,answers
0,"Наука - это деятельность, направленная на выра...","Наука - это деятельность, направленная на выра...",False,Какую цель преследует наука?,Наука преследует цель понимания и описания окр...
1,"Наука включает в себя сбор фактов, их регулярн...","Наука не включает в себя сбор фактов, их регул...",True,Какие этапы включает в себя наука при изучении...,"Наука включает в себя наблюдение, гипотезирова..."
2,"Научные гипотезы, которые описывают совокупнос...","Научные гипотезы, которые описывают совокупнос...",False,Как определяются научные законы?,Научные законы определяются путем повторения и...
3,Современная наука опирается на три основы: при...,Современная наука не опирается на три основы: ...,False,На что опирается современная наука?,Современная наука опирается на экспериментальн...
4,Наука в широком смысле включает в себя все усл...,Наука в широком смысле не включает в себя все ...,True,Что такое наука в широком смысле?,Наука в широком смысле — это систематическое и...
...,...,...,...,...,...
995,Педиатрия помогает детям реализовать свой поте...,"Педиатрия не включает в себя профилактическую,...",False,Какое значение имеет педиатрия для детского зд...,Диагностика и лечение психических расстройств
996,Педиатрия занимается методами лечения и сохран...,Педиатрия не помогает детям реализовать свой п...,False,Что включает в себя изучение педиатрии?,Кристиан Яков Лейден
997,Педиатрия изучает особенности детского организма.,Педиатрия не занимается методами лечения и сох...,False,Какая часть системы здравоохранения является п...,"Психические расстройства, их причины, симптомы..."
998,Педиатрия имеет важное значение для сохранения...,Педиатрия не изучает особенности детского орга...,False,Какая область клинической медицины изучает пси...,"Клиническая, экспериментальная и социальная пс..."


In [ ]:
file = input('Введите название файла:')+'.xlsx'
path ='/content/drive/MyDrive/kurswork/'
file_path = f'{path}/{file}'
df.to_excel(file_path, index=False)


Введите название файла:тест
